In [ ]:
from models.EEGAVI.base_EEGAVI.dataset import kd_train_dataset
from torch.utils.data import DataLoader
from models.EEGAVI.base_EEGAVI.EEGAVI import EEGAVI

model = EEGAVI(video_kd_size=100, audio_kd_size=100)
ds = kd_train_dataset("../../resources/AMIGOS/processed/spec.csv")